# MScFE 620 DERIVATIVE PRICING Group Work Project # 3

- The group is now asked to assess the causes of volatility by 2 properties: stochastic volatility and jumps in the underlying. Speciﬁcally, we will use the Heston Model and Merton Model instead of the Black-Scholes Model.

- To Do: This assignment is based on the same questions as in GWP1 and GWP2. To maintain the same numbering, questions begin from #5. Remember to use the following general parameters: S=80; r = 5.5%; σ = 35%; T = 3 months

- Also, remember that you will be employing Monte-Carlo method, so make sure you run a suﬃcient number of simulations and consider enough time steps (for you to determine).

# Step 1:
## Team Member A: Stochastic Volatility Modeler
For the Heston model, you can use the following parameters:
- $ν_0$ = 3. 2%
- $κ_ν$ = 1. 85
- $θ_ν$ = 0. 045

**5. Using the Heston Model and Monte-Carlo simulation, price an ATM European call and put, using a correlation value of -0.30.**

Using the Heston model and Monte Carlo simulation, we would initialize model and simulation parameters, generate random numbers for both asset and volatility processes, create a covariance matrix to account for correlation, simulate volatility paths, and then simulate asset price paths while taking the volatility paths into account in order to price an At-The-Money(ATM) European call and put option with a correlation value of -0.30. Next, we want to compute the call and put option payoffs at maturity, use the risk-free rate to discount these payoffs to present value, and average the outcomes of all the simulations. The anticipated values of the ATM European call and put options in the Heston model using Monte Carlo simulation are shown by the resulting average (Schumacher, 2020).

**An overview on the calculated estimates**

Based on the calculation presented below, the call has a value of $\$2.83$,  while the put is worth $\$2.83$ as well. European call and put options are frequently priced similarly, with the put options being marginally less expensive because of the opportunity cost of keeping cash rather than the underlying asset.

In [2]:
# importing all libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
from tabulate import tabulate

In [ ]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand
np.random.seed(42)

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80
r = 0.055
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 50000  # Number of simulations
dt = T / M  # Length of time step

rand = random_number_gen(M, Ite)
np.random.seed(42)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    avg = np.mean(payoff)

    return np.exp(-r * (T - t)) * avg

print("European Call Price under Heston model: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "C"))
print("European Put Price under Heston model: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "P"))

European Call Price under Heston model:  2.828469737881668
European Put Price under Heston model:  2.8327835614765298


In [ ]:
table_data = [
    ["5",  "European", "Call","Heston Model with Monte-Carlo simulation and a correlation value of -0.30", "$2.83"],
    ["5",  "European", "Put", "Heston Model with Monte-Carlo simulation and a correlation value of -0.30", "$2.83"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q_um",  "Exer","Type", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+----------+--------+---------------------------------------------------------------------------+---------+
|   Q_um | Exer     | Type   | GWP 3 Method                                                              | Price   |
+========+==========+========+===========================================================================+=========+
|      5 | European | Call   | Heston Model with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |
+--------+----------+--------+---------------------------------------------------------------------------+---------+
|      5 | European | Put    | Heston Model with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |
+--------+----------+--------+---------------------------------------------------------------------------+---------+


**6. Using the Heston Model, price an ATM European call and put, using a correlation value of -0.70**

The pricing of an ATM European call and put option with the call option valued at $\$2.1$ and the put option at $\$3.43$. This changes is significantly as the correlation value varies in the Heston model and it goes from -0.30 to -0.70. The main reason for this is how correlation affects the dynamics of option pricing. The cost of the call option decreases when the asset price rises because of the higher negative correlation (-0.70) between asset returns and volatility returns. On the other hand, the put option gains from the negative correlation, which raises the put option's price and protects against increased downside risk. This adjustment demonstrates the significance of correlation variations in influencing market sentiment, model calibration, and option price sensitivity(Schumacher, 2020).

In [ ]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand
np.random.seed(42)

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.70

S0 = 80
r = 0.055
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 10000
dt = T / M  # Length of time step

rand = random_number_gen(M, Ite)
np.random.seed(42)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

print("European Call Price under Heston model: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "C"))
print("European Put Price under Heston model: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "P"))

European Call Price under Heston model:  2.100269052154384
European Put Price under Heston model:  3.4294332097648033


In [ ]:
table_data = [
    ["6",  "European","Call", "Heston Model with a correlation value of -0.70", "$2.1"],
    ["6",  "European","Put", "Heston Model with a correlation value of -0.70", "$3.43"]
]

table_title = "Summary of Results"

headers = ["Q #s", "Exer","Type",  "GWP 3 Method", "Price"]

table = tabulate(table_data, headers=headers, tablefmt="grid")

table_with_title = f"{table_title}\n\n{table}"

print(table_with_title)

Summary of Results

+--------+----------+--------+------------------------------------------------+---------+
|   Q #s | Exer     | Type   | GWP 3 Method                                   | Price   |
+========+==========+========+================================================+=========+
|      6 | European | Call   | Heston Model with a correlation value of -0.70 | $2.1    |
+--------+----------+--------+------------------------------------------------+---------+
|      6 | European | Put    | Heston Model with a correlation value of -0.70 | $3.43   |
+--------+----------+--------+------------------------------------------------+---------+


**7. Calculate delta and gamma for each of the options in Questions 5 and 6. (Hint:
You can numerically approximate this by forcing a change in the variable of
interest –i.e., underlying stock price and delta change—and recalculate the option
price).**

"Delta" and "gamma" are Greek letters that stand for two significant risk sensitivity of options in the context of the Heston Model and Monte Carlo simulation for option pricing (SICA, F., DI LORENZO, E. M. I. L. I. A., & CUOMO, S., 2020).

The term "delta" refers to an option's price sensitivity to changes in the price of the underlying asset. By modeling a slight change in the asset price, reevaluating the option's price in each simulation, and computing the average change, delta can be computed in the context of the Heston Model and Monte Carlo simulation. For call options, it can have values between -1 and 1, and for put options, it can have values between -1 and 0. We determine delta to be 0.56 for the call option and 0.70 for the put option in the context of the Heston Model and Monte Carlo simulation with a rho of -0.30 (Schumacher, 2020).

On the other land, Gamma quantifies how quickly the delta of an option fluctuates in response to shifts in the price of the underlying asset. The calculation of gamma in the Heston Model and Monte Carlo simulation involves simulating slight variations in the asset price and the related delta changes. The second derivative of the option price with respect to the asset price, denoted as $(Γ = ∂^²V/∂S^²)$, is then determined. Because of this, gamma can be either positive or negative. Usually, it can be particularly detrimental for long option holdings. The delta for the call option is -0.43 and for the put option is -0.54, respectively, in the framework of the Heston Model and Monte Carlo simulation with a rho of -0.30 (Schumacher, 2020).

In [ ]:
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045
sigma_v = 0.35
rho = -0.30

# Option Parameters
S0 = 80
K_call = 80
K_put = 80
r = 0.055
T = 3/12  # Time to maturity (in years)
M0 = 50   # Number of time steps in a year
M = int(M0 * T)  # Total time steps
Ite = 50000
dt = T / M  # Length of time step

# Small perturbation for delta calculation
delta_S = 0.01

# Generate random numbers
rand = np.random.standard_normal((2, M + 1, Ite))
np.random.seed(42)

# Covariance Matrix
covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Function to simulate volatility paths
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1] + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

# Function to simulate stock price paths
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)
    return S

# Calculate option prices at the original stock price
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

call_price_original = np.mean(np.maximum(0, S[-1] - K_call) * np.exp(-r * T))
put_price_original = np.mean(np.maximum(0, K_put - S[-1]) * np.exp(-r * T))

# Calculate option prices with a slightly increased stock price
S_perturbed = S0 * (1 + delta_S)
V_perturbed = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S_perturbed = Heston_paths(S_perturbed, r, V_perturbed, 0, cho_matrix)

call_price_perturbed = np.mean(np.maximum(0, S_perturbed[-1] - K_call) * np.exp(-r * T))
put_price_perturbed = np.mean(np.maximum(0, K_put - S_perturbed[-1]) * np.exp(-r * T))

# Calculate delta and gamma for the European call and put options
call_delta = (call_price_perturbed - call_price_original) / (delta_S * S0)
put_delta = (put_price_perturbed - put_price_original) / (delta_S * S0)

# Calculate gamma using finite differences
call_gamma = ((call_price_perturbed - 2 * call_price_original + call_price_original) / (delta_S**2 * S0**2))
put_gamma = ((put_price_perturbed - 2 * put_price_original + put_price_original) / (delta_S**2 * S0**2))

# Print the results
print("ATM European Call Option Delta:", call_delta)
print("ATM European Call Option Gamma:", call_gamma)
print("ATM European Put Option Delta:", put_delta)
print("ATM European Put Option Gamma:", put_gamma)

ATM European Call Option Delta: 0.5573553338046949
ATM European Call Option Gamma: 0.6966941672558686
ATM European Put Option Delta: -0.42893484286741
ATM European Put Option Gamma: -0.5361685535842625


In [ ]:
table_data = [
    ["7",  "European", "Call","Heston Model with Monte-Carlo simulation and a correlation value of -0.30", "$2.83", "0.56", "0.70"],
    ["7",  "European","Put", "Heston Model with Monte-Carlo simulation and a correlation value of -0.30", "$2.83", "-0.43", "-0.54"]
]

# Define the title for the table
table_title = "Summary of Results for Delta and Gamma of each options in Question 5"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price", "Delta", "Gamma"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results for Delta and Gamma of each options in Question 5

+--------+----------+--------+---------------------------------------------------------------------------+---------+---------+---------+
|   Q #s | Type     | Exer   | GWP 3 Method                                                              | Price   |   Delta |   Gamma |
+========+==========+========+===========================================================================+=========+=========+=========+
|      7 | European | Call   | Heston Model with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |    0.56 |    0.7  |
+--------+----------+--------+---------------------------------------------------------------------------+---------+---------+---------+
|      7 | European | Put    | Heston Model with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |   -0.43 |   -0.54 |
+--------+----------+--------+---------------------------------------------------------------------------+--

## Team Member B: Jump Modeler
Using Merton model which uses Poisson distribution to model the extreme move of the underlying asset.

**8. Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.75**

Given parameters:

*   mu = -0.5
*   lambda = 0.75
*   delta = 0.22
*   Other inputs are the same as Question 5 above

We first model the distribution of underlying asset which accounts for extreme moves over the life of the option using Monte-Carlo methods. Then we average discounted payoff to derive the option price





In [ ]:
lamb = 0.75  # Lambda of the model
mu = -0.5  # Mu
delta = 0.22  # Delta
r = 0.055  # Risk-free rate
sigma = 0.35  # Volatility
T = 0.25 # Maturity/time period (in years)
S0 = 80  # Current Stock Price
Ite = 100000  # Number of simulations (paths)
M = 50  # Number of steps
dt = T / M  # Time-step

In [ ]:
SM = np.zeros((M + 1, Ite))
SM[0] = S0

# rj
rj = lamb * (np.exp(mu + 0.5 * delta**2) - 1)

# Random numbers
z1 = np.random.standard_normal((M + 1, Ite))
z2 = np.random.standard_normal((M + 1, Ite))
y = np.random.poisson(lamb * dt, (M + 1, Ite))


In [ ]:
for t in range(1, M + 1):
    SM[t] = SM[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SM[t] = np.maximum(
        SM[t], 0.00001)

In [ ]:
def merton_call_mc(S, K, r, T, t):
    payoff = np.maximum(0, S[-1, :] - K)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
def merton_put_mc(S, K, r, T, t):
    payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

In [ ]:
print("European Call Price under Merton model with 0.75 jump intensity: ",round(merton_call_mc(SM, 80, r, T, 0),2))
print("European Put Price under Merton model with 0.75 jump intensity: ", round(merton_put_mc(SM, 80, r, T, 0),2))

European Call Price under Merton model with 0.75 jump intensity:  8.28
European Put Price under Merton model with 0.75 jump intensity:  7.21


In [ ]:
table_dataq8 = [
    ["8",  "European", "Call","ATM European Call Price under Merton model with 0.75 jump intensity", "$8.23"],
    ["8",  "European", "Put", "ATM European Put Price under Merton model with 0.75 jump intensity", "$7.35"]
]

# Define the title for the table
table_titleq8 = "Summary of Results"

# Define the table headers
headersq8 = ["Q_no",  "Exer","Type", "GWP 3 Method", "Price"]

# Format the table using tabulate
tableq8 = tabulate(table_dataq8, headers=headersq8, tablefmt="grid")

# Add the title above the table
table_with_titleq8 = f"{table_titleq8}\n\n{tableq8}"

# Print the formatted table
print(table_with_titleq8)

Summary of Results

+--------+----------+--------+---------------------------------------------------------------------+---------+
|   Q_no | Exer     | Type   | GWP 3 Method                                                        | Price   |
+========+==========+========+=====================================================================+=========+
|      8 | European | Call   | ATM European Call Price under Merton model with 0.75 jump intensity | $8.23   |
+--------+----------+--------+---------------------------------------------------------------------+---------+
|      8 | European | Put    | ATM European Put Price under Merton model with 0.75 jump intensity  | $7.35   |
+--------+----------+--------+---------------------------------------------------------------------+---------+


**9. Using the Merton Model, price an ATM European call and put with jump intensity
parameter equal to 0.25.**

Parameters are the same with question 8 but using jump intensity of 0.25 instead

We first model the distribution of underlying asset which accounts for extreme moves over the life of the option using Monte-Carlo methos. Then we average discounted payoff to derive the option price

In [ ]:
lamb1 = 0.25
rj1 = lamb1 * (np.exp(mu + 0.5 * delta**2) - 1)
y1 = np.random.poisson(lamb1 * dt, (M + 1, Ite))
SM1 = np.zeros((M + 1, Ite))
SM1[0] = S0

for t in range(1, M + 1):
    SM1[t] = SM1[t - 1] * (
        np.exp((r - rj1 - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y1[t]
    )
    SM1[t] = np.maximum(
        SM1[t], 0.00001)
print("European Call Price under Merton model with 0.25 jump intensity: ", merton_call_mc(SM1, 80, r, T, 0))
print("European Put Price under Merton model with 0.25 jump intensity: ", merton_put_mc(SM1, 80, r, T, 0))

European Call Price under Merton model with 0.25 jump intensity:  6.7971969430588155
European Put Price under Merton model with 0.25 jump intensity:  5.717192539740826


In [ ]:
table_dataq9 = [
    ["9",  "European", "Call","ATM European Call Price under Merton model with 0.25 jump intensity", "$6.80"],
    ["9",  "European", "Put", "ATM European Put Price under Merton model with 0.25 jump intensity", "$5.72"]
]

# Define the title for the table
table_titleq9 = "Summary of Results"

# Define the table headers
headersq9 = ["Q_no",  "Exer","Type", "GWP 3 Method", "Price"]

# Format the table using tabulate
tableq9 = tabulate(table_dataq9, headers=headersq9, tablefmt="grid")

# Add the title above the table
table_with_titleq9 = f"{table_titleq9}\n\n{tableq9}"

# Print the formatted table
print(table_with_titleq9)

Summary of Results

+--------+----------+--------+---------------------------------------------------------------------+---------+
|   Q_no | Exer     | Type   | GWP 3 Method                                                        | Price   |
+========+==========+========+=====================================================================+=========+
|      9 | European | Call   | ATM European Call Price under Merton model with 0.25 jump intensity | $6.80   |
+--------+----------+--------+---------------------------------------------------------------------+---------+
|      9 | European | Put    | ATM European Put Price under Merton model with 0.25 jump intensity  | $5.72   |
+--------+----------+--------+---------------------------------------------------------------------+---------+


**10. Calculate delta and gamma for each of the options in Questions 8 and 9. (Hint:
You can use the same trick as in Question 7).**

We could approximate delta and gamma for option by calculating the option price and delta with respect to changing in underlying asset by small amount

In [ ]:
SMup = np.zeros((M + 1, Ite))
SMup[0] = S0+0.1
for t in range(1, M + 1):
    SMup[t] = SMup[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SMup[t] = np.maximum(
        SMup[t], 0.00001)
SMdown = np.zeros((M + 1, Ite))
SMdown[0] = S0-0.1
for t in range(1, M + 1):
    SMdown[t] = SMdown[t - 1] * (
        np.exp((r - rj - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y[t]
    )
    SMdown[t] = np.maximum(
        SMdown[t], 0.00001)
q8callpriceup = merton_call_mc(SMup, 80, r, T, 0)
q8callpricemean = merton_call_mc(SM, 80, r, T, 0)
q8callpricedown = merton_call_mc(SMdown, 80, r, T, 0)
q8deltacall = (q8callpriceup-q8callpricemean)/0.1
q8gammacall = (q8callpriceup-2*q8callpricemean+q8callpricedown)/0.1**2
q8putpriceup = merton_put_mc(SMup, 80, r, T, 0)
q8putpricemean = merton_put_mc(SM, 80, r, T, 0)
q8putpricedown = merton_put_mc(SMdown, 80, r, T, 0)
q8deltaput = (q8putpriceup-q8putpricemean)/0.1
q8gammaput = (q8putpriceup-2*q8putpricemean+q8putpricedown)/0.1**2
print("Delta of call option in question 8: ",q8deltacall)
print("Gamma of call option in question 8: ",q8gammacall)
print("Delta of put option in question 8: ",q8deltaput)
print("Gamma of put option in question 8: ",q8gammaput)

Delta of call option in question 8:  0.6494402918627173
Gamma of call option in question 8:  0.023569712321425126
Delta of put option in question 8:  -0.35034592180675617
Gamma of put option in question 8:  0.023569712321780397


In [ ]:
SM1up = np.zeros((M + 1, Ite))
SM1up[0] = S0+0.1
for t in range(1, M + 1):
    SM1up[t] = SM1up[t - 1] * (
        np.exp((r - rj1 - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y1[t]
    )
    SM1up[t] = np.maximum(
        SM1up[t], 0.00001)
SM1down = np.zeros((M + 1, Ite))
SM1down[0] = S0-0.1
for t in range(1, M + 1):
    SM1down[t] = SM1down[t - 1] * (
        np.exp((r - rj1 - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z1[t])
        + (np.exp(mu + delta * z2[t]) - 1) * y1[t]
    )
    SM1down[t] = np.maximum(
        SM1down[t], 0.00001)
q9callpriceup = merton_call_mc(SM1up, 80, r, T, 0)
q9callpricemean = merton_call_mc(SM1, 80, r, T, 0)
q9callpricedown = merton_call_mc(SM1down, 80, r, T, 0)
q9deltacall = (q9callpriceup-q9callpricemean)/0.1
q9gammacall = (q9callpriceup-2*q9callpricemean+q9callpricedown)/0.1**2
q9putpriceup = merton_put_mc(SM1up, 80, r, T, 0)
q9putpricemean = merton_put_mc(SM1, 80, r, T, 0)
q9putpricedown = merton_put_mc(SM1down, 80, r, T, 0)
q9deltaput = (q9putpriceup-q9putpricemean)/0.1
q9gammaput = (q9putpriceup-2*q9putpricemean+q9putpricedown)/0.1**2
print("Delta of call option in question 9: ",q9deltacall)
print("Gamma of call option in question 9: ",q9gammacall)
print("Delta of put option in question 9: ",q9deltaput)
print("Gamma of put option in question 9: ",q9gammaput)

Delta of call option in question 9:  0.598358593085857
Gamma of call option in question 9:  0.02557898432860028
Delta of put option in question 9:  -0.4015727459013885
Gamma of put option in question 9:  0.025578984328777917


In [ ]:
table_dataq10 = [
    ["10",  "European", "Call","ATM European Call Price under Merton model with 0.75 jump intensity", "0.65", "0.02"],
    ["10",  "European","Put", "ATM European Put Price under Merton model with 0.75 jump intensity", "-0.35", "0.02"],
    ["10",  "European", "Call","ATM European Call Price under Merton model with 0.25 jump intensity", "0.59", "0.02"],
    ["10",  "European", "Put","ATM European Put Price under Merton model with 0.25 jump intensity","-0.40", "0.02"]
]

# Define the title for the table
table_titleq10 = "Summary of Results for Delta and Gamma of each options in Question 8 & 9"

# Define the table headers
headersq10 = ["Q #s", "Type", "Exer", "GWP 3 Method", "Delta", "Gamma"]

# Format the table using tabulate
tableq10 = tabulate(table_dataq10, headers=headersq10, tablefmt="grid")

# Add the title above the table
table_with_titleq10 = f"{table_titleq10}\n\n{tableq10}"

# Print the formatted table
print(table_with_titleq10)

Summary of Results for Delta and Gamma of each options in Question 8 & 9

+--------+----------+--------+---------------------------------------------------------------------+---------+---------+
|   Q #s | Type     | Exer   | GWP 3 Method                                                        |   Delta |   Gamma |
+========+==========+========+=====================================================================+=========+=========+
|     10 | European | Call   | ATM European Call Price under Merton model with 0.75 jump intensity |    0.65 |    0.02 |
+--------+----------+--------+---------------------------------------------------------------------+---------+---------+
|     10 | European | Put    | ATM European Call Price under Merton model with 0.75 jump intensity |   -0.35 |    0.02 |
+--------+----------+--------+---------------------------------------------------------------------+---------+---------+
|     10 | European | Call   | ATM European Call Price under Merton model with 

# Team Member C: Model Validator

**11. For Questions 5, 6, 8, and 9, use put-call Parity to determine if the prices of the
put and call from the Heston Model and Merton Model satisfy put-call parity.**

- The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$, for Question 5 ,
As we can see
 $C_{0}= 2.83, -Ke^{-rT} = -78.91, S_{0} = 80, P_{0}= 2.83$.
So that $C_{0}=-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Heston Model.


- The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$, for Question 6 ,
As we can see
 $C_{0}= 2.1, -Ke^{-rT} = -78.91, S_{0} = 80,P_{0}= 3.43$.
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Heston Model.

- The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$, for Question 8 ,
As we can see
 $C_{0}= 8.23, -Ke^{-rT} = -78.91, S_{0} = 80,P_{0}=7.35$ .
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Merton Model.

- The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$, for Question 9 ,
As we can see
 $C_{0}=6.80, -Ke^{-rT} = -78.91, S_{0} = 80, P_{0}=5.72$ .
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Merton Model.

**12. Run the Heston Model and Merton Model for 7 different strikes: 3 OTM calls; 1
ATM call; and 3 ITM calls. The strikes should be equally spaced. Try to use the
following APPROXIMATE moneyness values: 0.85, 0.90, 0.95, 1, 1.05, 1.10, and
1.15. Recall that moneyness = stock/strike.**

In [ ]:
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

moneyness_values = [0.85, 0.90, 0.95, 1.0, 1.05, 1.10, 1.15]

# Calculate strikes based on moneyness values
strikes = [S0 * m for m in moneyness_values]

# Initialize lists to store option prices
call_option_prices = []

# Calculate option prices for each strike
for strike in strikes:
    call_price = heston_call_mc(S, strike, r, T, 0, "C")
    call_option_prices.append(call_price)
    print(f"Strike Price: {strike}, Call Option Price: {call_price:.2f}")
for strike in strikes:
    put_price = heston_call_mc(S, strike, r, T, 0, "P")
    call_option_prices.append(put_price)
    print(f"Strike Price: {strike}, Put Option Price: {put_price:.2f}")

Strike Price: 68.0, Call Option Price: 12.08
Strike Price: 72.0, Call Option Price: 8.48
Strike Price: 76.0, Call Option Price: 5.32
Strike Price: 80.0, Call Option Price: 2.86
Strike Price: 84.0, Call Option Price: 1.30
Strike Price: 88.0, Call Option Price: 0.51
Strike Price: 92.0, Call Option Price: 0.18
Strike Price: 68.0, Put Option Price: 0.20
Strike Price: 72.0, Put Option Price: 0.55
Strike Price: 76.0, Put Option Price: 1.33
Strike Price: 80.0, Put Option Price: 2.82
Strike Price: 84.0, Put Option Price: 5.20
Strike Price: 88.0, Put Option Price: 8.36
Strike Price: 92.0, Put Option Price: 11.98


In [ ]:
table_data = [
    ["12",  "European","Call", "Heston Model in combination with a correlation value of -0.30", "68","12.08"],
    ["12",  "European","Call", "Heston Model in combination with a correlation value of -0.30", "72","8.48"],
    ["12",  "European","Call", "Heston Model in combination with a correlation value of -0.30", "76","5.32"],
    ["12",  "European","Call", "Heston Model in combination with a correlation value of -0.30", "80","2.86"],
    ["12",  "European","Call", "Heston Model in combination with a correlation value of -0.30", "84","1.30"],
    ["12",  "European", "Call","Heston Model in combination with a correlation value of -0.30", "88","0.51"],
    ["12",  "European", "Call","Heston Model in combination with a correlation value of -0.30","92", "0.18"],
     ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30", "68","0.20"],
    ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30", "72","0.55"],
    ["12", "European","Put",  "Heston Model in combination with a correlation value of -0.30", "76","1.33"],
    ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30", "80","2.82"],
    ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30", "84","5.20"],
    ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30", "88","8.36"],
    ["12",  "European","Put", "Heston Model in combination with a correlation value of -0.30","92", "11.98"],
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method","Strikes", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+----------+--------+---------------------------------------------------------------+-----------+---------+
|   Q #s | Type     | Exer   | GWP 3 Method                                                  |   Strikes |   Price |
+========+==========+========+===============================================================+===========+=========+
|     12 | European | Call   | Heston Model in combination with a correlation value of -0.30 |        68 |   12.08 |
+--------+----------+--------+---------------------------------------------------------------+-----------+---------+
|     12 | European | Call   | Heston Model in combination with a correlation value of -0.30 |        72 |    8.48 |
+--------+----------+--------+---------------------------------------------------------------+-----------+---------+
|     12 | European | Call   | Heston Model in combination with a correlation value of -0.30 |        76 |    5.32 |
+--------+----------+--------+--------------

In [4]:
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 10000
n_steps = 100
np.random.seed(0)

def merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    jump_intensity = lamb * (np.exp(muJ + 0.5 * deltaJ**2) - 1)

    paths = np.zeros((n_steps + 1, n_paths))
    paths[0] = S0

    for t in range(1, n_steps + 1):
        normal_shocks = np.random.normal(0, 1, n_paths)
        jump_shocks = np.random.normal(muJ, deltaJ, n_paths)
        jump_sizes = np.random.poisson(lamb * dt, n_paths)

        paths[t] = paths[t-1] * np.exp((r - 0.5 * sigma**2 - jump_intensity) * dt + sigma * np.sqrt(dt) * normal_shocks + jump_shocks * jump_sizes)

    return paths

def european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)
    # Payoff at maturity for European call option
    payoff = np.maximum(paths[-1] - K, 0)
    # Discount the payoff to present value
    return np.mean(payoff) * np.exp(-r * T)

def european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * T)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Payoff at maturity for a European put
    payoff = np.maximum(K - paths[-1], 0)

    return np.mean(payoff) * discount



european_call_price = european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Call Option Price with jump intensity of 0.75: {european_call_price:.2f}")

european_put_price = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Put Option Price jump intensity of 0.75: {european_put_price:.2f}")

European Call Option Price with jump intensity of 0.75: 8.24
European Put Option Price jump intensity of 0.75: 7.40


In [6]:
table_data = [
    ["12",  "European Call Option Price with jump intensity of 0.75", "8.24"],
    ["12",  "European Put Option Price with jump intensity of 0.75", "7.40"]
]

# Define the title for the table
table_title = "Summary of Results for Delta and Gamma of each options in Question 5"

# Define the table headers
headers = ["Q #s", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results for Delta and Gamma of each options in Question 5

+--------+--------------------------------------------------------+---------+
|   Q #s | GWP 3 Method                                           |   Price |
+========+========================================================+=========+
|     12 | European Call Option Price with jump intensity of 0.75 |    8.24 |
+--------+--------------------------------------------------------+---------+
|     12 | European Put Option Price with jump intensity of 0.75  |    7.4  |
+--------+--------------------------------------------------------+---------+


In [ ]:
np.random.seed(0)

moneyness_values = [0.85, 0.90, 0.95, 1.0, 1.05, 1.10, 1.15]

# Calculate strikes based on moneyness values
strikes = [S0 * m for m in moneyness_values]

# Initialize lists to store option prices
call_option_prices = []

# Calculate option prices for each strike
for strike in strikes:
    european_call_price = european_call_merton(S0, strike, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    call_option_prices.append(european_call_price)
    print(f"Strike Price: {strike}, Call Option Price: {european_call_price:.2f}")
for strike in strikes:
    european_put_price = european_put_merton(S0, strike, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    call_option_prices.append(european_put_price)
    print(f"Strike Price: {strike}, Put Option Price: {european_put_price:.2f}")

Strike Price: 68.0, Call Option Price: 16.29
Strike Price: 72.0, Call Option Price: 13.30
Strike Price: 76.0, Call Option Price: 10.65
Strike Price: 80.0, Call Option Price: 8.26
Strike Price: 84.0, Call Option Price: 6.27
Strike Price: 88.0, Call Option Price: 4.68
Strike Price: 92.0, Call Option Price: 3.39
Strike Price: 68.0, Put Option Price: 3.33
Strike Price: 72.0, Put Option Price: 4.36
Strike Price: 76.0, Put Option Price: 5.66
Strike Price: 80.0, Put Option Price: 7.14
Strike Price: 84.0, Put Option Price: 9.19
Strike Price: 88.0, Put Option Price: 11.47
Strike Price: 92.0, Put Option Price: 14.19


In [ ]:
table_data = [
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "68","16.29"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "72","13.40"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "76","10.63"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "80","8.24"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "84","6.33"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "88","4.69"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "92","3.38"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "68","3.40"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "72","4.36"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "76","5.56"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "80","7.22"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "84","9.14"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "88","11.55"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "92","14.17"],
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method","Strikes" "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+----+--------+----------+---------------------------------------------------------+----------------+----------------+
|    | Q #s   | Type     | Exer                                                    |   GWP 3 Method |   StrikesPrice |
+====+========+==========+=========================================================+================+================+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             68 |          16.29 |
+----+--------+----------+---------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             72 |          13.4  |
+----+--------+----------+---------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             76 |          10.63 |
+----+--------+----------+--

#Step 2: All team members:

**Question 13: Repeat Questions 5 and 8 for the case of an American call option.**

13. Repeat question 8 for American call option as below. We iterate backward through time steps, calculating the intrinsic value, discounted future values and early exercise value at each step. Then we compare if early exercise has greater value

In [11]:

S0 = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 100000
n_steps = 100

def american_call_option(S, K, r, T, t):
    # Initialize the option value at maturity
    option_value = np.maximum(0, S[-1, :] - K)

    # Time step
    dt = T / len(S)

    # Perform backward induction
    for i in range(len(S) - 2, -1, -1):
        # Calculate the present value of option value
        option_value = np.exp(-r * dt) * (
            option_value  # Continue holding the option
        )

        # Calculate the intrinsic value (exercise value) of the option
        exercise_value = np.maximum(0, S[i, :] - K)

        # Update the option value if it's optimal to exercise
        option_value = np.maximum(option_value, exercise_value)

    # Calculate the average option value across all simulations
    option_price = np.mean(option_value)

    return option_price

american_call_price = american_call_option(S, 80, 0.055, 3/12, 0)
print(f"American Call Price under Heston: , {american_call_price:.2f}")


American Call Price under Heston: , 4.59


In [ ]:
table_data = [
    ["5", "Call", "European", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "2.83"],
    ["13", "Call", "American", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "4.59"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q Nums", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+----------+--------+----------+-------------------------------------------------------------------------------------------+---------+
|   Q Nums | Type   | Exer     | GWP 3 Method                                                                              |   Price |
+==========+========+==========+===========================================================================================+=========+
|        5 | Call   | European | Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30 |    2.83 |
+----------+--------+----------+-------------------------------------------------------------------------------------------+---------+
|       13 | Call   | American | Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30 |    4.59 |
+----------+--------+----------+-------------------------------------------------------------------------------------------+---------+


**Comment on the differences you observe from original Questions 5**

We must implement the early exercise feature of American options in order to obtain an American call option. In order to accomplish this, we incorporate a backward induction loop that determines if exercising the option at each time step prior to maturity is desirable. For a variety of reasons, including the use of Monte Carlo simulations in the framework of the Heston model, the cost of an American call option is therefore typically higher than the cost of a European call option ([analystprep.com](https://analystprep.com/study-notes/frm/part-1/financial-markets-and-products/properties-of-stock-options/)).  This is caused by route dependence, stochastic volatility, the early exercise option, and the best time to execute. When it comes to early exercise options, European call options can only be performed at expiration, but American call options allow the holder to exercise the option at any time before or at expiration. We call this optional early exercise. This early exercise clause makes American options more advantageous than other options since it allows the option holder to profit from favorable price fluctuations in the underlying asset. American call options are generally more expensive because of their greater flexibility (Marroni & Perdomo, 2014).

The concept of optimal exercise time also emphasizes the significance of timing at every stage of simulation while pricing an American option using Heston model simulations. This means comparing the intrinsic value of the option—that is, the difference between the strike price and the current asset price—with the calculated option value. If the intrinsic value of the American call option is higher than the option value, then it makes sense to exercise it. Option pricing may increase when comparing this feature to European options, which lack this flexibility (Marroni & Perdomo, 2014).

Finally, the Heston model adds stochastic volatility to path-dependence and stochastic volatility, which increases the dynamic nature of the option pricing process. This path-dependent feature of American options is advantageous because it allows the option holder to exercise at times of favorable volatility, potentially raising option values. Conversely, European options simply rely on the asset price at expiration and are not affected by a route. In the Heston model, path-dependence and stochastic volatility combine to further explain why American call options are priced higher (Marroni & Perdomo, 2014).

**Pricing American all using Mertons model:**

In [5]:
def merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    jump_intensity = lamb * (np.exp(muJ + 0.5 * deltaJ**2) - 1)

    paths = np.zeros((n_steps + 1, n_paths))
    paths[0] = S0

    for t in range(1, n_steps + 1):
        normal_shocks = np.random.normal(0, 1, n_paths)
        jump_shocks = np.random.normal(muJ, deltaJ, n_paths)
        jump_sizes = np.random.poisson(lamb * dt, n_paths)

        paths[t] = paths[t-1] * np.exp((r - 0.5 * sigma**2 - jump_intensity) * dt + sigma * np.sqrt(dt) * normal_shocks + jump_shocks * jump_sizes)

    return paths

def american_option_lsm(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * dt)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Initialize payoffs at maturity
    payoff = np.maximum(paths[-1] - K, 0)

    for t in range(n_steps - 1, 0, -1):
        # Select in-the-money paths
        itm = paths[t] > K
        X = paths[t, itm]
        Y = payoff[itm] * discount

        # Using polynomial regression for continuation value estimation
        reg = np.polyfit(X, Y, 2)
        continuation = np.polyval(reg, X)

        # Update the payoff
        immediate_exercise = np.maximum(X - K, 0)
        payoff[itm] = np.where(immediate_exercise > continuation, immediate_exercise, payoff[itm] * discount)

    return np.mean(payoff) * discount


S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 100000
n_steps = 50

american_call_price = american_option_lsm(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"American Call Option Price Using mertons model is: ${american_call_price:.2f}")

American Call Option Price Using mertons model is: $8.36


In [ ]:
table_data = [
    ["5", "Call", "European", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "2.83"],
    ["13", "Call", "American", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "4.59"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q Nums", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

**Comment on differences observed from original question 8**

- Sensitivity to Jump Parameters: Jump parameters such as the volatility $(δ)$, mean $(μ)$, and intensity $(λ)$ of leaps are introduced by the Merton model. Compared to European options, the value of American options will be more susceptible to these factors because of its early exercise feature, especially when the likelihood of jumps impacting future prices is high..

- Complexity of Computation:
 The Merton model arises from the need to consider optimal exercise decisions at each time step, the interaction with jump dynamics, path-dependent features, stochastic volatility, and the use of numerical methods. These factors collectively contribute to a more intricate and resource-intensive pricing process, especially when dealing with American options. Calculating the best stopping times in the presence of jumps requires additional computational effort compared to European options, where exercise decisions are made only at expiration. The interaction between jump dynamics and the early exercise feature, which is absent in European options, contributes to the computational intricacy.

- Effect of Jumping on Early Exercise: Big upward increases (skewed to the right/positive skewness) could make it less desirable to exercise American calls at an early age. If there's a good likelihood that prices will rise in the future,

**14. Using Heston model data from Question 6, price a European up-and-in call option
(UAI) with a barrier level of $\$95$ and a strike price of $\$95$ as well. This UAI option
becomes alive only if the stock price reaches (at some point before maturity) the
barrier level (even if it ends below it). Compare the price obtained to the one from
the simple European call.**




Because the DAI put option requires additional requirements before it becomes active, its price is often lower than the price of a standard ATM European put option. These additional requirements are as follows (Marroni & Perdomo, 2014):

 (i) it is only activated (alive) if the underlying stock price crosses or touches a predefined barrier level during the option's lifetime. If the stock price remains above the barrier level throughout the option's life, the DAI option expires worthless;

(ii) the barrier level is typically set below the initial stock price, which means the stock price must move against the holder's favor to activate the option.DAI put options are less expensive than a standard ATM European put option because of these features that reduce the likelihood that they will become active and pay out. The reduced price corresponds to a decreased likelihood of profit for the DAI option (Marroni & Perdomo, 2014).


In contrast, an ATM European put option pays out if the stock price at expiration is less than the strike price and has no barrier restrictions. It usually has a greater price since it has a higher probability of being lucrative. The risk-reward trade-off connected with barrier options, such as the DAI put option, is reflected in the price differential between these two options. If the stock prices decline, investors who assumed additional risk that the option may never expire receive compensation ([analystprep.com](https://analystprep.com/study-notes/frm/part-1/financial-markets-and-products/properties-of-stock-options/).)

In [ ]:
# Heston Model Parameters
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045
sigma_v = 0.35
rho = -0.70  # Updated correlation value

# Option Parameters
S0 = 80
K = 95
B = 95
r = 0.055
T = 3/12  # Time to maturity (in years)
M0 = 50   # Number of time steps in a year
M = int(M0 * T)  # Total time steps
Ite = 100000  # Number of simulations
dt = T / M  # Length of time step

# Generate random numbers
rand = np.random.standard_normal((2, M + 1, Ite))

# Covariance Matrix
covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Function to simulate volatility paths
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M
    v = np.zeros((M + 1, Ite), dtype=float)
    v[0] = v0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1] + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

# Function to simulate stock price paths
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)
    return S

# Calculate option prices
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

# Initialize variables to track UAI call option price
uai_call_price = 0

# Iterate through simulation paths
for i in range(Ite):
    barrier_crossed = False  # Flag to check if barrier is crossed

    for t in range(1, M + 1):
        if S[t, i] >= B:
            barrier_crossed = True
            break  # Barrier crossed, exit inner loop

    if barrier_crossed:
        uai_call_price += np.maximum(0, S[-1, i] - K) * np.exp(-r * T)  # Calculate UAI call option price

# Calculate the average UAI call option price
uai_call_price /= Ite

# Print the UAI call option price
print("European Up-and-In Call Option Price:", uai_call_price)

European Up-and-In Call Option Price: 0.005827337456274561


In [13]:
table_data = [
    ["6", "European","Call", "ATM European Call for a simple Heston Model", "2.1"],
    ["14",  "American", "Call","Up-and-in Call option (UAI) with a barrier level/strike price $95", "0.01"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s",  "Exer","Type", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+----------+--------+-------------------------------------------------------------------+---------+
|   Q #s | Exer     | Type   | GWP 3 Method                                                      |   Price |
+========+==========+========+===================================================================+=========+
|      6 | European | Call   | ATM European Call for a simple Heston Model                       |    2.1  |
+--------+----------+--------+-------------------------------------------------------------------+---------+
|     14 | American | Call   | Up-and-in Call option (UAI) with a barrier level/strike price $95 |    0.01 |
+--------+----------+--------+-------------------------------------------------------------------+---------+


**15: Using Merton model data from Question 8, price a European down-and-in put
option (DAI) with a barrier level of 65 and a strike price of 65 as well. This UAO
option becomes alive only if the stock price reaches (at some point before
maturity) the barrier level (even if it ends above it).
We first check across the path if barrier breached. If not breached, pay off should be equal 0**

In [ ]:
def merton_down_and_in_put_mc(S, K, B, r, T, t):
    # Check if the barrier is breached during the option's life
    barrier_breached = np.any(S < B, axis=0)

    # Calculate the payoff for the down-and-in put option
    payoff = np.where(barrier_breached, np.maximum(0, K - S[-1, :]), 0)

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average
print("DAI Option Price using Merton with 0.75 jump intensity:", merton_down_and_in_put_mc(SM, 65, 65, r, T, 0))

DAI Option Price using Merton with 0.75 jump intensity: 2.7979467627558656


In [ ]:
table_dataq15 = [
    ["8",  "European", "Put","ATM European Call Price under Merton model with 0.75 jump intensity", "$7.35"],
    ["15",  "DAI", "Put", "DAI Put Price under Merton model with 0.75 jump intensity", "$2.80"]
]

# Define the title for the table
table_titleq15 = "Summary of Results"

# Define the table headers
headersq15 = ["Q_no",  "Exer","Type", "GWP 3 Method", "Price"]

# Format the table using tabulate
tableq15 = tabulate(table_dataq15, headers=headersq15, tablefmt="grid")

# Add the title above the table
table_with_titleq15 = f"{table_titleq15}\n\n{tableq15}"

# Print the formatted table
print(table_with_titleq15)

Summary of Results

+--------+----------+--------+---------------------------------------------------------------------+---------+
|   Q_no | Exer     | Type   | GWP 3 Method                                                        | Price   |
+========+==========+========+=====================================================================+=========+
|      8 | European | Put    | ATM European Call Price under Merton model with 0.75 jump intensity | $7.35   |
+--------+----------+--------+---------------------------------------------------------------------+---------+
|     15 | DAI      | Put    | DAI Put Price under Merton model with 0.75 jump intensity           | $2.80   |
+--------+----------+--------+---------------------------------------------------------------------+---------+


Compare the price obtained to the one from the simple European put:
Option price of DAI type is much lower than the one in question 8 which reflects the harder condition for the option to be alive and the barrier is quite far away from the initial underlying price. The European Put Option Price with Jump Intensity of 0.75 at Strike $65, as we can see, is substantially smaller than the European Down-and-In Put Option Price with barrier.

#Reference

- Hsu, Y. L. et al. "Constant Elasticity of Variance (CEV) Option Pricing Model: Integration and Detailed Variation." Mathematics and Computers in Simulation, vol. 79, no. 1, 2008, pp. 60–71.
- SICA, F., DI LORENZO, E. M. I. L. I. A., & CUOMO, S. Meshless Methods for Option Pricing and Risks Computation.

- Schumacher, J. M. (2020). Introduction to Financial Derivatives: Modeling, Pricing and Hedging. Open Press TiU: https://digi-courses.com/openpresstiu-introduction-to-financial-derivatives/

- Marroni L. & I. Perdomo (2014). Pricing and Hedging Financial Derivatives: A Guide for Practitioners. Wiley

- https://analystprep.com/study-notes/frm/part-1/financial-markets-and-products/properties-of-stock-options/